In [2]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PeiyingWu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import re
r'\nRevision: (\d+)\n'
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [42]:
import pandas as pd
import csv
data = pd.read_fwf('C:/Users/PeiyingWu/Desktop/Fordham/Spring/Text analytics/Project/Tesla2019.txt',sep="delimter" ,engine='python',header=None)
data.head()



,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,"Tesla, Inc. (0001318605)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ITEM 1.\tBUSINESS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Overview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"We design, develop, manufacture and sell high-...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
##rename column name
data.columns
data.rename(columns={0:"text"},inplace=True)

In [9]:
data["text"]

0                               Tesla, Inc. (0001318605)
1                                      ITEM 1.\tBUSINESS
2                                                    NaN
3                                               Overview
4      We design, develop, manufacture and sell high-...
5      We currently produce and sell three fully elec...
6      We commenced deliveries of Model S in June 201...
7      We also intend to bring additional all-electri...
8      •\tLong Range and Recharging Flexibility. Our ...
9      •\tHigh-Performance Without Compromised Design...
10     •\tEnergy Efficiency and Cost of Ownership. Ou...
11     We sell our vehicles through our own sales and...
12                                                     1
13              ________________________________________
14                                                   NaN
15     In addition, we are leveraging our technologic...
16     Finally, we sell and lease solar energy system...
17     We manufacture our vehic

In [12]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data["text"]))

print(data_words)

[['tesla', 'inc'], ['item', 'business'], ['nan'], ['overview'], ['we', 'design', 'develop', 'manufacture', 'and', 'sell', 'high', 'performance', 'fully', 'electric', 'vehicles', 'evs', 'and', 'energy', 'generation', 'and', 'storage', 'systems', 'and', 'also', 'install', 'and', 'maintain', 'such', 'energy', 'systems', 'and', 'sell', 'solar', 'electricity', 'we', 'are', 'the', 'world', 'first', 'vertically', 'integrated', 'sustainable', 'energy', 'company', 'offering', 'end', 'to', 'end', 'clean', 'energy', 'products', 'including', 'generation', 'storage', 'and', 'consumption', 'we', 'have', 'established', 'and', 'continue', 'to', 'grow', 'global', 'network', 'of', 'stores', 'galleries', 'vehicle', 'service', 'centers', 'mobile', 'service', 'technicians', 'body', 'shops', 'supercharger', 'stations', 'and', 'destination', 'chargers', 'to', 'accelerate', 'the', 'widespread', 'adoption', 'of', 'our', 'products', 'and', 'we', 'continue', 'to', 'develop', 'self', 'driving', 'capability', 'in'

In [17]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words]])

In [23]:

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [26]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized)

[['tesla', 'inc'], ['item', 'business'], ['nan'], ['overview'], ['design', 'develop', 'manufacture', 'sell', 'high', 'performance', 'fully', 'electric', 'vehicle', 'evs', 'energy', 'generation', 'storage', 'system', 'also', 'install', 'maintain', 'energy', 'system', 'sell', 'solar', 'electricity', 'world', 'first', 'vertically', 'integrate', 'sustainable', 'energy', 'company', 'offering', 'end', 'end', 'clean', 'energy', 'product', 'include', 'generation', 'storage', 'consumption', 'establish', 'continue', 'grow', 'global', 'network', 'store', 'gallery', 'vehicle', 'service', 'center', 'mobile', 'service', 'technician', 'body', 'shop', 'supercharger', 'station', 'destination', 'charger', 'accelerate', 'widespread', 'adoption', 'product', 'continue', 'develop', 'self_drive', 'capability', 'order', 'improve', 'vehicle', 'safety', 'sustainable', 'energy', 'product', 'engineering', 'expertise', 'intense', 'focus', 'accelerate', 'world', 'transition', 'sustainable', 'energy', 'business', 'm

In [28]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus)

[[(0, 1), (1, 1)], [(2, 1), (3, 1)], [(4, 1)], [(5, 1)], [(2, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 2), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 2), (24, 6), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 3), (51, 1), (52, 1), (53, 2), (54, 2), (55, 1), (56, 1), (57, 1), (58, 2), (59, 1), (60, 1), (61, 3), (62, 2), (63, 1), (64, 1), (65, 3), (66, 1), (67, 1), (68, 2)], [(21, 1), (31, 1), (36, 1), (45, 3), (49, 1), (53, 1), (65, 3), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1)], [(16, 2), (17, 1), (30, 1), (37, 1), (45, 6), (49, 1), (71, 1), (72, 1), (74, 1), (79, 1), (80, 1), (81, 2), (82, 3), (83, 1), (84, 4), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1)

In [29]:
id2word[0]

'inc'

In [31]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus]

[[('inc', 1), ('tesla', 1)],
 [('business', 1), ('item', 1)],
 [('nan', 1)],
 [('overview', 1)],
 [('business', 1),
  ('accelerate', 2),
  ('adoption', 1),
  ('also', 1),
  ('body', 1),
  ('capability', 1),
  ('center', 1),
  ('charger', 1),
  ('clean', 1),
  ('company', 2),
  ('consumption', 1),
  ('continue', 2),
  ('design', 1),
  ('destination', 1),
  ('develop', 2),
  ('differentiate', 1),
  ('electric', 1),
  ('electricity', 1),
  ('end', 2),
  ('energy', 6),
  ('engineering', 1),
  ('establish', 1),
  ('evs', 1),
  ('expertise', 1),
  ('first', 1),
  ('focus', 1),
  ('fully', 1),
  ('gallery', 1),
  ('generation', 2),
  ('global', 1),
  ('grow', 1),
  ('high', 1),
  ('improve', 1),
  ('include', 1),
  ('install', 1),
  ('integrate', 1),
  ('intense', 1),
  ('maintain', 1),
  ('manufacture', 1),
  ('mobile', 1),
  ('model', 1),
  ('network', 1),
  ('offering', 1),
  ('order', 1),
  ('performance', 1),
  ('product', 3),
  ('safety', 1),
  ('self_drive', 1),
  ('sell', 2),
  ('serv

In [33]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [34]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.041*"vehicle" + 0.038*"model" + 0.028*"production" + '
  '0.015*"manufacturing" + 0.014*"manufacture" + 0.013*"certain" + '
  '0.013*"cost" + 0.012*"plan" + 0.012*"gigafactory" + 0.011*"high"'),
 (1,
  '0.024*"supplier" + 0.023*"may" + 0.020*"equipment" + 0.017*"make" + '
  '0.016*"information" + 0.014*"right" + 0.012*"license" + 0.011*"cause" + '
  '0.011*"product" + 0.011*"intellectual_property"'),
 (2,
  '0.036*"may" + 0.032*"could" + 0.017*"business" + 0.016*"product" + '
  '0.016*"result" + 0.013*"market" + 0.012*"cost" + 0.012*"future" + '
  '0.012*"impact" + 0.011*"include"'),
 (3,
  '0.034*"common_stock" + 0.019*"price" + 0.017*"share" + '
  '0.017*"convertible_note" + 0.016*"tesla" + 0.016*"certain" + 0.015*"could" '
  '+ 0.014*"may" + 0.012*"term" + 0.010*"note"'),
 (4,
  '0.035*"energy" + 0.026*"system" + 0.022*"solar" + 0.019*"customer" + '
  '0.018*"state" + 0.018*"vehicle" + 0.017*"service" + 0.015*"nan" + '
  '0.015*"storage" + 0.013*"sell"')]


In [41]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


C:\Users\peiywu\Downloads\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.164663  0.156696       1        1  31.163458
2      0.001735 -0.247164       2        1  25.459263
0      0.081506  0.009688       3        1  22.400793
3     -0.279853  0.092185       4        1  11.303065
1      0.031950 -0.011404       5        1   9.673424, topic_info=     Category        Freq              Term       Total  loglift  logprob
term                                                                     
24    Default  143.000000            energy  143.000000  30.0000  30.0000
45    Default  122.000000             model  122.000000  29.0000  29.0000
1064  Default  125.000000             could  125.000000  28.0000  28.0000
871   Default  176.000000               may  176.000000  27.0000  27.0000
1628  Default   49.000000      common_stock   49.000000  26.0000  26.0000
100   Default   88.000000        production   88.000000  25.0000  25.0000
56    Default   88.000000             solar   88.000000  24.0000  24.0000
62    Default  113.000000            system  113.000000  23.0000  23.0000
54    Default   68.000000           service   68.000000  22.0000  22.0000
65    Default  233.000000           vehicle  233.000000  21.0000  21.0000
118   Default   77.000000          customer   77.000000  20.0000  20.0000
636   Default   29.000000          supplier   29.000000  19.0000  19.0000
267   Default   75.000000             state   75.000000  18.0000  18.0000
169   Default   64.000000           certain   64.000000  17.0000  17.0000
4     Default   58.000000               nan   58.000000  16.0000  16.0000
924   Default   34.000000         equipment   34.000000  15.0000  15.0000
135   Default   68.000000            result   68.000000  14.0000  14.0000
99    Default   59.000000             price   59.000000  13.0000  13.0000
94    Default   48.000000     manufacturing   48.000000  12.0000  12.0000
609   Default   31.000000              make   31.000000  11.0000  11.0000
651   Default   25.000000             share   25.000000  10.0000  10.0000
1795  Default   24.000000  convertible_note   24.000000   9.0000   9.0000
690   Default   48.000000          warranty   48.000000   8.0000   8.0000
2     Default   71.000000          business   71.000000   7.0000   7.0000
58    Default   65.000000           storage   65.000000   6.0000   6.0000
643   Default   40.000000            impact   40.000000   5.0000   5.0000
545   Default   20.000000       information   20.000000   4.0000   4.0000
1106  Default   30.000000           license   30.000000   3.0000   3.0000
279   Default   35.000000       gigafactory   35.000000   2.0000   2.0000
43    Default   51.000000       manufacture   51.000000   1.0000   1.0000
...       ...         ...               ...         ...      ...      ...
1248   Topic5    4.043420          exchange    4.731343   2.1787  -5.7182
638    Topic5    3.874220              test    4.548763   2.1753  -5.7610
1325   Topic5    3.846171        acceptable    4.522395   2.1738  -5.7682
1671   Topic5    3.997909     vulnerability    4.726377   2.1684  -5.7295
1729   Topic5    3.480325        litigation    4.150793   2.1596  -5.8682
1698   Topic5    3.468243            decide    4.151037   2.1561  -5.8717
159    Topic5    3.325118      acceleration    3.987123   2.1542  -5.9138
1236   Topic5   17.680052             right   21.272934   2.1508  -4.2429
214    Topic5    6.077497       application    7.481260   2.1280  -5.3107
924    Topic5   24.449823         equipment   34.622192   1.9879  -3.9187
609    Topic5   20.782036              make   31.874538   1.9081  -4.0812
249    Topic5    7.482371            instal    9.889364   2.0569  -5.1028
1162   Topic5    5.252977      depreciation    6.530101   2.1182  -5.4565
1051   Topic5   11.100526            report   16.781498   1.9225  -4.7083
713    Topic5   13.302312             cause   22.138489   1.8264  -4.5274
216    Topic5    9.372115  